In [ ]:
import json
import numpy as np
import h5py
import tensorflow as tf
import os
import cPickle as pkl
#import ipdb
import numpy
import copy
import pprint
from Recognizer import MathFormulaRecognizer
import warnings
import sys
import math
import time
from util import *
from collections import OrderedDict

from data_iterator import dataIterator
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
home_path = os.getcwd()
checkpoint_path = os.path.join(home_path,'save','model.ckpt')
checkpoint_dir  =os.path.join(home_path,'save')
max_iters = 100000
batch_size = 16
valid_batch_size = 8
finetune_encoder_after = -1
#Evaluation Checkpoint
nEvaImages = 300
EvaEach = 2500
SummaryEach = 1000
device = "/gpu:0"
batch_Imagesize=250000
valid_batch_Imagesize=500000
maxImagesize = 500000
maxlen = 200
n_epoch = 10000

In [ ]:
#build model
model = MathFormulaRecognizer(num_label =112,dim_hidden=128,device = '/device:GPU:0')
loss,opt = model.build_train()
saver = tf.train.Saver(max_to_keep=10)
sess = tf.Session()
saved_path=tf.train.latest_checkpoint(checkpoint_dir)
start_step = 0
if(saved_path):
    tf.reset_default_graph()
    saver.restore(sess, saved_path)
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir) 
    step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
    start_step = int(step)
    print('model restored',start_step)
else:
    sess.run(tf.global_variables_initializer())


In [ ]:
datasets=['./data/offline-train.pkl',
'./data/train_caption.txt']
valid_datasets=['./data/offline-test.pkl',
'./data/test_caption.txt']
dictionaries=['./data/dictionary.txt']


worddicts = load_dict(dictionaries[0])
worddicts_r = [None] * len(worddicts)

for kk, vv in worddicts.iteritems():
        worddicts_r[vv] = kk
train,train_uid_list = dataIterator(datasets[0], datasets[1],
                         worddicts,
                         batch_size=batch_size, batch_Imagesize=batch_Imagesize,maxlen=maxlen,maxImagesize=maxImagesize)
valid,valid_uid_list = dataIterator(valid_datasets[0], valid_datasets[1],
                     worddicts,
                     batch_size=valid_batch_size, batch_Imagesize=valid_batch_Imagesize,maxlen=maxlen,maxImagesize=maxImagesize)


In [ ]:
train = np.squeeze(train)
valid = np.squeeze(valid)
n_train_img = train.shape[0]
n_valid_img = valid.shape[0]

for i in range(start_step/n_train_img,n_epoch):
    rand_permute = np.arange(n_train_img)
    np.random.shuffle(rand_permute)
    saver.save(sess, checkpoint_path, global_step=i*rand_permute.shape[0])
    avg_loss = 0
    count = 0
    print('epoch: ',i)
    for j in range(0,rand_permute.shape[0]):
        x, x_mask, y, y_mask = prepare_data(train[rand_permute[j],0],train[rand_permute[j],1])
        y = np.transpose(y)
        y_mask = np.transpose(y_mask)
        _,Loss = sess.run([opt,loss],feed_dict={model.x:x,model.x_mask:x_mask,model.y:y,\
                                                model.y_mask:y_mask,model.is_train:True,model.lr:0.002})
        avg_loss = avg_loss + Loss
        count = count + 1
        if(not(j %100)):
            avg_loss = avg_loss / count
            print(j,avg_loss)
            count = 0
            avg_loss = 0

In [ ]:

# train = np.squeeze(train)
# valid = np.squeeze(valid)
# n_train_img = train.shape[0]
# n_valid_img = valid.shape[0]


In [ ]:
# print(x_entropy)
# plt.imshow(x_mask[0,:,:],cmap='gray')

# # for i in range(0,len(train[800,0])):
# #     print(train[800,0][i].shape)
# im = train[0,0][2]
# im = im.reshape(im.shape[1],im.shape[2])
# plt.imshow(im,cmap='gray')

In [ ]:
# print(logit.shape)
# np.argmax(logit,-1)